# Semantic Segmentation

In [1]:
%load_ext tensorboard

import math
import time
import datetime
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

import semantic_segmentation.models as models
import semantic_segmentation.ops.datasets as datasets
import semantic_segmentation.ops.imageops as imageops
import semantic_segmentation.ops.trains as trains
import semantic_segmentation.ops.tests as tests

In [2]:
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
plt.rcParams["figure.figsize"] = (4, 4)
plt.rcParams["font.size"] = 15
plt.rcParams["figure.titlesize"] = 25
plt.rcParams["axes.labelsize"] = 20
plt.rcParams["xtick.labelsize"] = 15
plt.rcParams["ytick.labelsize"] = 15
plt.rcParams["legend.fontsize"] = 13
plt.rcParams["lines.linewidth"] = 2

# plt.rcParams["font.family"] = "serif"
# plt.rcParams["mathtext.fontset"] = "dejavuserif"
# plt.rcParams["figure.figsize"] = (8, 8)
# plt.rcParams["font.size"] = 30
# plt.rcParams["axes.labelsize"] = 48
# plt.rcParams["xtick.labelsize"] = 40
# plt.rcParams["ytick.labelsize"] = 40
# plt.rcParams["legend.fontsize"] = 28
# plt.rcParams["lines.linewidth"] = 4

Parameters:

In [3]:
# Paths
cwd = os.getcwd()
model_path = "models_checkpoints"
dat_path = "leaderboard/semantic-segmentation"

# CamVid
dataset_name = 'camvid'
img_size = 720 // 2, 960 // 2
crop_size = 720 // 2, 960 // 2
dataset_root = '%s/datasets/camvid' % cwd
seq_root = 'F:/research/dataset/camvid/seq'

# CityScape
# dataset_name = 'cityscape'
# img_size = 1024 // 2, 2048 // 2
# crop_size = 480, 560
# dataset_root = 'F:/research/dataset/cityscape'
# seq_root = 'F:/research/dataset/cityscape'


Load label informations:

In [4]:
colors = datasets.colors(dataset_name)
num_classes = len(set(colors.values()))
print("%d classes are loaded. " % num_classes)

11 classes are loaded. 


In [6]:
offset = (10, 0)
dataset_train, dataset_val, dataset_test = datasets.dataset(
    dataset_name, dataset_root, img_size, crop_size)
dataset_seq = datasets.dataset_seq(
    dataset_name, dataset_root, seq_root, img_size, offset=offset)
dataset_test = dataset_seq.map(lambda image, label: (image[offset[0] - 1], label))

In [7]:
# class_weights = datasets.median_freq_weights(dataset_train, num_classes)  # manually calculates weights
class_weights = datasets.memorized_median_freq_weights(dataset_name)  # load memorized weights
print("Class weights: \n", class_weights.numpy())

Class weights: 
 [0.30734012 0.19833793 4.7175865  0.16562003 0.6806351  0.42397258
 4.2133756  3.256359   1.         6.7325764  9.058633  ]


In [8]:
# DNN
model = models.UNet(num_classes, name='u-net-dnn')
# model = models.SegNet(num_classes, name='segnet-dnn')

# BNN
# model = models.UNet(num_classes, rate=0.5, name='u-net-bnn')
# model = models.SegNet(num_classes, rate=0.5, name='segnet-bnn')

# model.load_weights("%s/%s_%s" % (model_path, dataset_name, model.name))

TensorBoard variables:

In [9]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/gradient_tape/%s_%s/%s' % (dataset_name, model.name, current_time)
train_log_dir = '%s/train' % log_dir
test_log_dir = '%s/test' % log_dir
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
    
print('Create TensorBoard Log dir: ', log_dir)

Create TensorBoard Log dir:  logs/gradient_tape/camvid_u-net-dnn/20200801-165813


## A. Train 

In [ ]:
epochs = 300
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.9, beta_2=0.999)

epochtime_metric = tf.keras.metrics.Mean(name='epoch_time')
loss_metric = tf.keras.metrics.Mean(name='train_loss')
nll_metric = tf.keras.metrics.Mean(name='train_nll')

for epoch in range(epochs):
    batch_time = time.time()
    loss, nll = trains.train_epoch(optimizer, model, dataset_train, num_classes, class_weights, batch_size=3)
    epochtime_metric(time.time() - batch_time)
    loss_metric(loss)
    nll_metric(nll)
    
    if (epoch + 1) % 1 == 0:
        template = '(%.2f sec) Epoch: %d, Loss: %.4f, NLL: %.4f'
        print(template % (epochtime_metric.result(),
                          epoch,
                          loss_metric.result(),
                          nll_metric.result()))
        
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', loss_metric.result(), step=epoch)
            tf.summary.scalar('nll', nll_metric.result(), step=epoch)
        
        epochtime_metric.reset_states()
        loss_metric.reset_states()
        nll_metric.reset_states()

    if (epoch + 1) % 2 == 0:
        metrics = tests.test_sampling(model, 10, 
                                      dataset_test, num_classes, 
                                      batch_size=3, cutoffs=(0.0, 0.9), verbose=False)
        
        with test_summary_writer.as_default():
            tf.summary.scalar('nll', metrics[0], step=epoch)
            tf.summary.scalar('iou', metrics[2][0], step=epoch)
            tf.summary.scalar('iou-90', metrics[2][1], step=epoch)
            tf.summary.scalar('acc', metrics[3][0], step=epoch)
            tf.summary.scalar('acc-90', metrics[3][1], step=epoch)
            tf.summary.scalar('unc-90', metrics[4][1], step=epoch)
            tf.summary.scalar('cov-90', metrics[5][1], step=epoch)
            tf.summary.scalar('ece', metrics[9], step=epoch)
            tf.summary.image('calibration diagrams', metrics[10], step=epoch)


In [ ]:
model.save_weights("%s/%s_%s" % (model_path, dataset_name, model.name))

## B. Test

In [ ]:
_ = tests.test_vanilla(model, 
                       dataset_test, num_classes, batch_size=3, cutoffs=(0.0, 0.7, 0.9), verbose=True)

In [ ]:
_ = tests.test_sampling(model, 10, 
                        dataset_test, num_classes, batch_size=3, cutoffs=(0.0, 0.7, 0.9), verbose=True)

In [ ]:
_ = tests.test_temporal_smoothing(model, 1.0, 
                                  dataset_seq, num_classes, batch_size=3, cutoffs=(0.0, 0.7, 0.9), verbose=True)

In [ ]:
och_x_params = {'k': 10, 'l': 1.3, 's': 1.0}
och_y_params = {'k': 10, 'l': 1.3, 's': 1.0}
_ = tests.test_vq(model, och_x_params, och_y_params, 
                  dataset_seq, num_classes, cutoffs=(0.0, 0.7, 0.9), verbose=True)

In [ ]:
for xs, ys in dataset_test.shuffle(1).batch(1).take(1):
    ys_pred = tests.predict_sampling(model, xs, 10)
    ys_pred, unc_pred = tf.math.argmax(ys_pred, axis=-1), tf.math.reduce_max(ys_pred, axis=-1)
    
    ys = imageops.to_color(ys, colors)
    ys_pred = imageops.to_color(ys_pred, colors)
    fig, axes = plt.subplots(1, 4, figsize=(22, 5))
    for ax, image in zip(axes, [xs[0], ys[0], ys_pred[0], unc_pred[0]]):
        ax.imshow(image, cmap='gray')
        ax.axis('off')
    

In [ ]:
for xs, ys in dataset_seq.shuffle(1).batch(1).take(1):
    ys_pred = tests.predict_temporal_smoothing(model, xs, 1.0)
    ys_pred, unc_pred = tf.math.argmax(ys_pred, axis=-1), tf.math.reduce_max(ys_pred, axis=-1)
    
    ys = imageops.to_color(ys, colors)
    ys_pred = imageops.to_color(ys_pred, colors)
    fig, axes = plt.subplots(1, 4, figsize=(22, 5))
    for ax, image in zip(axes, [xs[0, -1], ys[0], ys_pred[0], unc_pred[0]]):
        ax.imshow(image, cmap='gray')
        ax.axis('off')
    